Fran Mička, Matej Lopotar, Jakov Samardžija

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import datetime
from sklearn import svm
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split 
from scipy.signal import butter, filtfilt
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import math

In [2]:
file = open ('StudentGrades.txt')
all_lines = []
midterm1_points = dict()
midterm2_points = dict()
final_points = dict()
for line in file.readlines():
    if (line.strip() != '' and '---' not in line):
        all_lines.append (line.strip())
for i in range (1, 11): 
    x = all_lines[i].split (' – ')
    midterm1_points['S'+str(i)] = int(x[1])
    
for i in range (12, 22): 
    x = all_lines[i].split (' – ')
    midterm2_points['S'+str(i-11)] = int(x[1])
    
for i in range (23, 33): 
    x = all_lines[i].split (' – ')
    final_points['S'+str(i-22)] = int(x[1])
    
print (midterm1_points)
print (midterm2_points)
print (final_points)

{'S1': 78, 'S2': 82, 'S3': 77, 'S4': 75, 'S5': 67, 'S6': 71, 'S7': 64, 'S8': 92, 'S9': 80, 'S10': 89}
{'S1': 82, 'S2': 85, 'S3': 90, 'S4': 77, 'S5': 77, 'S6': 64, 'S7': 33, 'S8': 88, 'S9': 39, 'S10': 64}
{'S1': 182, 'S2': 180, 'S3': 188, 'S4': 149, 'S5': 157, 'S6': 175, 'S7': 110, 'S8': 184, 'S9': 126, 'S10': 116}


In [3]:
midterm1_beginning = {
    'S1': 46650,
    'S2': 47300,
    'S3': 45292,
    'S4': 46235,
    'S5': 48080,
    'S6': 44500,
    'S7': 45304,
    'S8': 46190,
    'S9': 46677,
    'S10': 47950
}


midterm2_beginning = {
    'S1': 66650,
    'S2': 66650,
    'S3': 66650,
    'S4': 66650,
    'S5': 66650,
    'S6': 66650,
    'S7': 66650,
    'S8': 66650,
    'S9': 66650,
    'S10': 66650
}

final_beginning = {
    'S1': 35808,
    'S2': 36130,
    'S3': 1265,
    'S4': 24880,
    'S5': 1282,
    'S6': 44114,
    'S7': 37090,
    'S8': 23962,
    'S9': 47787,
    'S10': 47950
}

In [4]:
def lowpass_butter_filter(data, cutoff, sampling_frequency, order=4):
    nyq = 0.5 * sampling_frequency
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    y = filtfilt(b, a, data, axis=0)
    return pd.DataFrame(y, columns=[data.columns[0]])

In [5]:
cutoff_freq = pd.DataFrame(
    {
        'Midterm 1': [0.002, 0.002, 0.0002, 0.002, 0.001, 0.002, 0.001, 0.0002, 0.002, 0.002],
        'Midterm 2': [0.002, 0.001, 0.0002, 0.002, 0.001, 0.002, 0.0002, 0.002, 0.002, 0.002],
        'Final': [0.002, 0.002, 0.001, 0.002, 0.001, 0.002, 0.0002, 0.002, 0.002, 0.002],
    }
)
print(cutoff_freq)

   Midterm 1  Midterm 2   Final
0     0.0020     0.0020  0.0020
1     0.0020     0.0010  0.0020
2     0.0002     0.0002  0.0010
3     0.0020     0.0020  0.0020
4     0.0010     0.0010  0.0010
5     0.0020     0.0020  0.0020
6     0.0010     0.0002  0.0002
7     0.0002     0.0020  0.0020
8     0.0020     0.0020  0.0020
9     0.0020     0.0020  0.0020


In [74]:
def feature_engineer_EDA(test_type, time_read_min, cutoff_freq, acc_beginning):
    all_students = []
    remove_eda = []
    for i in range (1, 11):      
        acc_y = pd.read_csv('Data/S' + str(i) + '/' + test_type + '/ACC.csv')
        acc_y = acc_y[acc_y.columns[1]]
        acc_y = acc_y.iloc[1:]
        
        acc_y = acc_y.iloc[acc_beginning['S'+str(i)]]

        #racunanje koliko smo vremena odrezali iza acc_y
        acc_y_freq = 32
        eda_freq = 4
        time_accY = acc_beginning['S'+str(i)] / acc_y_freq
        #print(time_accY)

        #micanje podataka iz eda na temelju frekvencija i vremena
        remove_eda.append(math.floor(time_accY * eda_freq))
        #print(math.floor(time_accY * eda_freq))

        df = pd.read_csv ('Data/S' + str(i) + '/' + str(test_type) + '/EDA.csv')
        df = df.iloc[math.floor(time_accY * eda_freq):]
        cutoff = cutoff_freq.at[i-1, test_type]
        df = lowpass_butter_filter(df, cutoff, 4.000)
        data =[]
        for el in df.values:
            data.append (el)
            
        range_data = int(time_read_min*60*4)


        y1 = data[0:range_data]
        y2 = data[int(len(data)/2) - int(range_data/2) : int(len(data)/2) + int(range_data/2)]
        y3 = data[len(data) - range_data : len(data)]

        mi1 = np.mean (y1)
        mi2 = np.mean (y2)
        mi3 = np.mean (y3)

        sigma1 = np.var (y1)
        sigma2 = np.var (y2)
        sigma3 = np.var (y3)

        ro = mi2 / (mi1 + mi3)

        diff = np.array(y3) - np.array(y1)
        mi_diff = np.mean (diff)
        sigma_diff = np.var (diff)

        x = [mi1, mi2, mi3, sigma1, sigma2, sigma3, ro, mi_diff, sigma_diff]
        all_students.append(x)
    return pd.DataFrame(all_students), remove_eda

midterm1_5_eda, removeable_eda5 = feature_engineer_EDA ('Midterm 1', 5, cutoff_freq, midterm1_beginning)
midterm1_10_eda, removeable_eda10 = feature_engineer_EDA ('Midterm 1', 10, cutoff_freq, midterm1_beginning)
midterm1_15_eda, removeable_eda15 = feature_engineer_EDA ('Midterm 1', 15, cutoff_freq, midterm1_beginning)

midterm2_5_eda, removeable2_eda5 = feature_engineer_EDA ('Midterm 2', 5, cutoff_freq, midterm2_beginning)
midterm2_10_eda, removeable2_eda10 = feature_engineer_EDA ('Midterm 2', 10, cutoff_freq, midterm2_beginning)
midterm2_15_eda, removeable2_eda15 = feature_engineer_EDA ('Midterm 2', 15, cutoff_freq, midterm2_beginning)

final_5_eda, removeablef_eda5 = feature_engineer_EDA ('Final', 5, cutoff_freq, final_beginning)
final_10_eda, removeablef_eda10 = feature_engineer_EDA ('Final', 10, cutoff_freq, final_beginning)
final_15_eda, removeablef_eda15 = feature_engineer_EDA ('Final', 15, cutoff_freq, final_beginning)

Dodan feature engineering za podatke HR - Značajke su dobivene sličnom metodom kao iz članka

In [75]:
def feature_engineer_HR(test_type, time_read_min, cutoff_freq, acc_beginning):
    all_students = []
    remove_hr = []
    for i in range (1, 11):      
        acc_y = pd.read_csv('Data/S' + str(i) + '/' + test_type + '/ACC.csv')
        acc_y = acc_y[acc_y.columns[1]]
        acc_y = acc_y.iloc[1:]
        
        acc_y = acc_y.iloc[acc_beginning['S'+str(i)]]

        #racunanje koliko smo vremena odrezali iza acc_y
        acc_y_freq = 32
        hr_freq = 1
        time_accY = acc_beginning['S'+str(i)] / acc_y_freq
        #print(time_accY)

        #micanje podataka iz eda na temelju frekvencija i vremena
        remove_hr.append(math.floor(time_accY * hr_freq))
        #print(math.floor(time_accY * hr_freq))

        df = pd.read_csv ('Data/S' + str(i) + '/' + str(test_type) + '/HR.csv')
        df = df.iloc[math.floor(time_accY * hr_freq):]
        cutoff = cutoff_freq.at[i-1, test_type]
        df = lowpass_butter_filter(df, cutoff, 4.000)
        data =[]
        for el in df.values:
            data.append (el)
            
        range_data = int(time_read_min*60*hr_freq)


        y1 = data[0:range_data]
        y2 = data[int(len(data)/2) - int(range_data/2) : int(len(data)/2) + int(range_data/2)]
        y3 = data[len(data) - range_data : len(data)]

        mi1 = np.mean (y1)
        mi2 = np.mean (y2)
        mi3 = np.mean (y3)

        sigma1 = np.var (y1)
        sigma2 = np.var (y2)
        sigma3 = np.var (y3)

        ro = mi2 / (mi1 + mi3)

        diff = np.array(y3) - np.array(y1)
        mi_diff = np.mean (diff)
        sigma_diff = np.var (diff)

        x = [mi1, mi2, mi3, sigma1, sigma2, sigma3, ro, mi_diff, sigma_diff]
        all_students.append(x)
    return pd.DataFrame(all_students), remove_hr

midterm1_5_hr, removeable_hr5 = feature_engineer_HR ('Midterm 1', 5, cutoff_freq, midterm1_beginning)
midterm1_10_hr, removeable_hr10 = feature_engineer_HR ('Midterm 1', 10, cutoff_freq, midterm1_beginning)
midterm1_15_hr, removeable_hr15 = feature_engineer_HR ('Midterm 1', 15, cutoff_freq, midterm1_beginning)

midterm2_5_hr, removeable2_hr5 = feature_engineer_HR ('Midterm 2', 5, cutoff_freq, midterm2_beginning)
midterm2_10_hr, removeable2_hr10 = feature_engineer_HR ('Midterm 2', 10, cutoff_freq, midterm2_beginning)
midterm2_15_hr, removeable2_hr15 = feature_engineer_HR ('Midterm 2', 15, cutoff_freq, midterm2_beginning)

final_5_hr, removeablef_hr5 = feature_engineer_HR ('Final', 5, cutoff_freq, final_beginning)
final_10_hr, removeablef_hr10 = feature_engineer_HR ('Final', 10, cutoff_freq, final_beginning)
final_15_hr, removeablef_hr15 = feature_engineer_HR ('Final', 15, cutoff_freq, final_beginning)



Podjela studenata po bodovima (>= 80% klasa 1, < 80% klasa 0)

In [76]:
midterm1_y = []
midterm2_y = []
final_y = []

for el in midterm1_points:
    if (midterm1_points[el] >= 80):
        midterm1_y.append (1)
    else:
        midterm1_y.append (0)
        
for el in midterm2_points:
    if (midterm2_points[el] >= 80):
        midterm2_y.append (1)
    else:
        midterm2_y.append (0)

for el in final_points:
    if (final_points[el] >= 160):
        final_y.append (1)
    else:
        final_y.append (0)
        
y1 = pd.DataFrame (midterm1_y)
y2 = pd.DataFrame (midterm2_y)
y3 = pd.DataFrame (final_y)


In [84]:
def KNN(dataframe, y):
    kf = KFold(n_splits=10)
    score = []
    for i, (train_index, test_index) in enumerate(kf.split(dataframe)):
        x_train = dataframe.loc[train_index]
        x_test = dataframe.loc[test_index]
        y_train = y.loc[train_index]
        y_test = y.loc[test_index]
    
        neigh = KNeighborsClassifier(n_neighbors=3, algorithm = 'auto').fit(np.array(x_train), np.array(y_train).ravel())
        
        score.append(neigh.score(x_test, y_test))
    return (score.count(1.0)/len(score)) * 100

print ('KNN')
print ('Midterm 1, 5min, HR: ', KNN(midterm1_5_hr, y1))
print ('Midterm 1, 10min, HR: ', KNN(midterm1_10_hr, y1))
print ('KNN, Midterm 1, 15min, HR: ', KNN(midterm1_15_hr, y1))
print ()

print ('KNN')
print ('Midterm 2, 5min, HR: ', KNN(midterm2_5_hr, y2))
print ('Midterm 2, 10min, HR: ', KNN(midterm2_10_hr, y2))
print ('Midterm 2, 15min, HR: ', KNN(midterm2_15_hr, y2))
print ()

print ('KNN')
print ('Final, 5min, HR: ', KNN(final_5_hr, y3))
print ('Final, 10min, HR: ', KNN(final_10_hr, y3))
print ('Final, 15min, HR: ', KNN(final_15_hr, y3))
print ()

KNN
Midterm 1, 5min, HR:  20.0
Midterm 1, 10min, HR:  40.0
KNN, Midterm 1, 15min, HR:  50.0

KNN
Midterm 2, 5min, HR:  30.0
Midterm 2, 10min, HR:  10.0
Midterm 2, 15min, HR:  10.0

KNN
Final, 5min, HR:  80.0
Final, 10min, HR:  80.0
Final, 15min, HR:  60.0



In [85]:
print ('KNN')
print ('Midterm 1, 5min, EDA: ', KNN(midterm1_5_eda, y1))
print ('Midterm 1, 10min, EDA: ', KNN(midterm1_10_eda, y1))
print ('KNN, Midterm 1, 15min, EDA: ', KNN(midterm1_15_eda, y1))
print ()

print ('KNN')
print ('Midterm 2, 5min, EDA: ', KNN(midterm2_5_eda, y2))
print ('Midterm 2, 10min, EDA: ', KNN(midterm2_10_eda, y2))
print ('Midterm 2, 15min, EDA: ', KNN(midterm2_15_eda, y2))
print ()

print ('KNN')
print ('Final, 5min, EDA: ', KNN(final_5_eda, y3))
print ('Final, 10min, EDA: ', KNN(final_10_eda, y3))
print ('Final, 15min, EDA: ', KNN(final_15_eda, y3))
print ()

KNN
Midterm 1, 5min, EDA:  50.0
Midterm 1, 10min, EDA:  50.0
KNN, Midterm 1, 15min, EDA:  50.0

KNN
Midterm 2, 5min, EDA:  60.0
Midterm 2, 10min, EDA:  70.0
Midterm 2, 15min, EDA:  60.0

KNN
Final, 5min, EDA:  20.0
Final, 10min, EDA:  0.0
Final, 15min, EDA:  20.0



Spojene značajke: EDA + HR

In [78]:
midterm1_5_edahr = pd.concat([midterm1_5_eda, midterm1_5_hr], axis=1, ignore_index = True)
midterm1_10_edahr = pd.concat([midterm1_10_eda, midterm1_10_hr], axis=1, ignore_index = True )
midterm1_15_edahr = pd.concat([midterm1_15_eda, midterm1_15_hr], axis=1, ignore_index = True)
print (midterm1_5_edahr)

midterm2_5_edahr = pd.concat([midterm2_5_eda, midterm2_5_hr], axis=1, ignore_index = True)
midterm2_10_edahr = pd.concat([midterm2_10_eda, midterm2_10_hr], axis=1, ignore_index = True )
midterm2_15_edahr = pd.concat([midterm2_15_eda, midterm2_15_hr], axis=1, ignore_index = True)

final_5_edahr = pd.concat([final_5_eda, final_5_hr], axis=1, ignore_index = True)
final_10_edahr = pd.concat([midterm2_10_eda, final_10_hr], axis=1, ignore_index = True )
final_15_edahr = pd.concat([midterm2_15_eda, final_15_hr], axis=1, ignore_index = True)

         0         1         2             3         4             5   \
0  0.024120  0.149306  0.026671  7.464472e-09  0.000247  3.684383e-09   
1  0.011720  0.468654  0.012833  4.472922e-07  0.001789  5.393751e-09   
2  0.066030  0.174352  0.270007  3.529261e-04  0.000002  3.830674e-11   
3  0.002269  0.127926  0.000036  3.917019e-06  0.000003  3.716458e-11   
4  0.009844  0.171779 -0.001603  6.711810e-05  0.000056  2.833260e-10   
5 -0.000006  1.219622  0.002727  5.573067e-11  0.001933  3.142604e-08   
6 -0.000427  0.213105  0.000465  2.572327e-07  0.000125  1.458538e-11   
7  0.000658  0.755975  0.848322  1.063183e-05  0.000384  3.617160e-10   
8  0.013604  0.319585  0.014491  8.452696e-08  0.000192  3.177951e-09   
9  0.019435  0.359947  0.021409  3.649713e-08  0.000002  9.391109e-10   

            6         7             8           9           10          11  \
0     2.939624  0.002551  5.906112e-09  117.555093   91.813466  100.619711   
1    19.087423  0.001114  5.471918e-07  

In [79]:
print ('KNN')
print ('Midterm 1, 5min, EDA and HR: ', KNN(midterm1_5_edahr, y1))
print ('Midterm 1, 10min, EDA and HR: ', KNN(midterm1_10_edahr, y1))
print ('Midterm 1, 15min, EDA and HR: ', KNN(midterm1_15_edahr, y1))
print ()

print ('KNN')
print ('Midterm 2, 5min, EDA and HR: ', KNN(midterm2_5_edahr, y1))
print ('Midterm 2, 10min, EDA and HR: ', KNN(midterm2_10_edahr, y1))
print ('Midterm 2, 15min, EDA and HR: ', KNN(midterm2_15_edahr, y1))
print()

print ('KNN')
print ('Final, 5min, EDA and HR: ', KNN(final_5_edahr, y1))
print ('Final, 10min, EDA and HR: ', KNN(final_10_edahr, y1))
print ('Final, 15min, EDA and HR: ', KNN(final_15_edahr, y1))
print()

KNN
Midterm 1, 5min, EDA and HR:  30.0
Midterm 1, 10min, EDA and HR:  60.0
Midterm 1, 15min, EDA and HR:  40.0

KNN
Midterm 2, 5min, EDA and HR:  50.0
Midterm 2, 10min, EDA and HR:  60.0
Midterm 2, 15min, EDA and HR:  60.0

KNN
Final, 5min, EDA and HR:  30.0
Final, 10min, EDA and HR:  40.0
Final, 15min, EDA and HR:  40.0



In [80]:
#Midterm 1
X_pca = PCA(n_components = 2).fit_transform(np.array(midterm1_5_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('SVM')
print ('Midterm 1, 5min, EDA and HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(midterm1_10_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('Midterm 1, 10min, EDA and HR: ' + str(accuracy*100))


X_pca = PCA(n_components = 2).fit_transform(np.array(midterm1_15_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('Midterm 1, 15min, EDA and HR: ' + str(accuracy*100))




#Midterm 2
print ()
X_pca = PCA(n_components = 2).fit_transform(np.array(midterm2_5_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y2).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y2).ravel())
print ('SVM')
print ('Midterm 2, 5min, EDA and HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(midterm2_10_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y2).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y2).ravel())
print ('Midterm 2, 10min, EDA and HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(midterm2_15_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y2).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y2).ravel())
print ('Midterm 2, 15min, EDA and HR: ' + str(accuracy*100))



#Final
print ()
X_pca = PCA(n_components = 2).fit_transform(np.array(final_5_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y3).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y3).ravel())
print ('SVM')
print ('Final, 5min, EDA and HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(final_10_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y3).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y3).ravel())
print ('Final, 10min, EDA and HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(final_15_edahr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y3).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y3).ravel())
print ('Final, 15min, EDA and HR: ' + str(accuracy*100))


SVM
Midterm 1, 5min, EDA and HR: 60.0
Midterm 1, 10min, EDA and HR: 70.0
Midterm 1, 15min, EDA and HR: 70.0

SVM
Midterm 2, 5min, EDA and HR: 60.0
Midterm 2, 10min, EDA and HR: 60.0
Midterm 2, 15min, EDA and HR: 60.0

SVM
Final, 5min, EDA and HR: 70.0
Final, 10min, EDA and HR: 70.0
Final, 15min, EDA and HR: 70.0


In [83]:
#Midterm1
X_pca = PCA(n_components = 2).fit_transform(np.array(midterm1_5_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('SVM')
print ('Midterm 1, 5min, HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(midterm1_10_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('Midterm 1, 10min, HR: ' + str(accuracy*100))


X_pca = PCA(n_components = 2).fit_transform(np.array(midterm1_15_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('Midterm 1, 15min, HR: ' + str(accuracy*100))
print ()


#Midterm2
X_pca = PCA(n_components = 2).fit_transform(np.array(midterm2_5_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('SVM')
print ('Midterm 2, 5min, HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(midterm2_10_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('Midterm 2, 10min, HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(midterm2_15_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y1).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y1).ravel())
print ('Midterm 2, 15min, HR: ' + str(accuracy*100))


#Final
print()
X_pca = PCA(n_components = 2).fit_transform(np.array(final_5_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y3).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y3).ravel())
print ('SVM')
print ('Final, 5min, HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(final_10_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y3).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y3).ravel())
print ('Final, 10min, HR: ' + str(accuracy*100))

X_pca = PCA(n_components = 2).fit_transform(np.array(final_15_hr))
clf = svm.SVC(kernel='linear').fit (X_pca, np.array(y3).ravel())
y_pred = clf.predict(X_pca)
accuracy = clf.score(X_pca, np.array(y3).ravel())
print ('Final, 15min, HR: ' + str(accuracy*100))

SVM
Midterm 1, 5min, HR: 60.0
Midterm 1, 10min, HR: 60.0
Midterm 1, 15min, HR: 70.0

SVM
Midterm 2, 5min, HR: 60.0
Midterm 2, 10min, HR: 60.0
Midterm 2, 15min, HR: 60.0

SVM
Final, 5min, HR: 70.0
Final, 10min, HR: 70.0
Final, 15min, HR: 70.0
